In [1]:
import transformers
import torch
import os
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

/data/nlp/aditya/miniforge3/envs/lmharness/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define which model are you testing
model_name = "albert-xxlarge-v2"

In [3]:
def load_model(model_name, task_name):
    # Load the configuration
    config_path = f"MTL-E/{task_name}_model/{model_name}/config.json"
    config = transformers.AutoConfig.from_pretrained(config_path)

    # Load the model state dict
    model_path = f"MTL-E/{task_name}_model/{model_name}/pytorch_model.bin"
    model = transformers.AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_name_or_path=None,
        config=config,
        state_dict=torch.load(model_path)
    )

    # Load the tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(f"MTL-E/{task_name}_model/{model_name}")

    return model, tokenizer


In [4]:
# Function to load test data from CSV
def load_test_data(csv_path):
    data = pd.read_csv(csv_path)
    print(data)
    return data

In [5]:
def evaluate_model(model, tokenizer, test_data, batch_size=8):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    model.to(device)
    true_list = []
    pred_list = []
    
    for i in range(0, len(test_data), batch_size):
        batch_data = test_data.iloc[i:i+batch_size]
        inputs = tokenizer(batch_data['Sentence'].tolist(), max_length=128, padding=True, truncation=True, return_tensors="pt")
        labels = torch.tensor(batch_data['labels'].tolist())
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)
        
        with torch.no_grad():
            logits = model(**inputs).logits
        
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        true_list.extend(labels.cpu().numpy())
        pred_list.extend(predictions)
    
    # Calculate evaluation metrics
    conf_matrix = confusion_matrix(true_list, pred_list)
    class_report = classification_report(true_list, pred_list, output_dict=True)
    
    return conf_matrix, class_report

In [6]:
# Load models
bias_model, bias_tokenizer = load_model(model_name, "bias")
stereotype_model, stereotype_tokenizer = load_model(model_name, "stereotype")

# Evaluate models
for task_name, model, tokenizer in [("bias", bias_model, bias_tokenizer), ("stereotype", stereotype_model, stereotype_tokenizer)]:
    # Load test data
    #Give path to test data
    test_csv_path = "MTL-E/dataset/StereoSet/test.csv" if task_name=='stereotype' else "MTL-E/dataset/ToxicBias/test.csv"
    test_data = load_test_data(test_csv_path)
    print(f"Evaluating {task_name} model...")
    conf_matrix, class_report = evaluate_model(model, tokenizer, test_data)

    print("---------------------------------Confusion Matrix------------------------------------")
    print(pd.DataFrame(conf_matrix))

    print("---------------------------------Evaluation Metrics------------------------------------")
    print(pd.DataFrame(class_report).transpose())
        
    # Save results to CSV
    # result_csv_path = f"./results/{task_name}_{model_name}_results.csv"
    # pd.DataFrame(class_report).transpose().to_csv(result_csv_path, index=True)
    # print(f"Results saved to {result_csv_path}")

                                              Sentence  identity_attack_x  \
0     As someone pointed out earlier ISIS and radic...           0.800000   
1     URL bellamycharlottesvilletwitter  Read up on...           0.506024   
2    . . .  she wants to give other women of color ...           0.500000   
3    . The whole world hates the white world yet th...           0.500000   
4    ... what horrible human beings .... pathetic. ...           0.653333   
..                                                 ...                ...   
645  Youre right he should pick something else to b...           0.500000   
646  Youre right the international condemnation of ...           0.700000   
647  Youre the one making claims.  I called you on ...           0.600000   
648  Youre white so clearly you dont understand or ...           0.800000   
649  Yup I keep telling people people like you are ...           0.636364   

     labels          target       category  \
0         1  muslims,whites  